In [70]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime
from dateutil import relativedelta
import requests
import json
from pandas.io.json import json_normalize
import sqlite3

cnx = sqlite3.connect('price.db')

In [71]:
def url(symbol):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+symbol+'&outputsize=full&apikey=ZBTGX838US0G5RE2'
    return url

In [209]:
def checkSymbol(symbol):
    #url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords='+symbol+'&apikey=ZBTGX838US0G5RE2'
    #response = requests.get(url)
    #data = pd.DataFrame.from_dict(response.json()['bestMatches'])
    #return data['1. symbol']
    data = pd.read_sql('select * from symbol', cnx,index_col='index')
    if (data.empty):
        return 0
    else:
        if (data[data['symbol'] == symbol].empty):
            return 0 
        else:
            return 1

In [212]:
print(checkSymbol('AAPL'))

1


In [72]:
print(url('MSFT'))

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=MSFT&outputsize=full&apikey=ZBTGX838US0G5RE2


In [110]:
def getPriceData(symbol):
    try:
        price = pd.read_sql('select * from '+symbol, cnx,index_col='index')
        return price
    except: 
        try:
            price = retrieveDataFromAPI(symbol)
            price.to_sql(name=symbol, con=cnx)
            return price
        except:
            return 'symbol tidak ditemukan'

In [118]:
def retrieveDataFromAPI(symbol):
    response = requests.get(url(symbol))
    try:
        timeseries = response.json()['Time Series (Daily)']
        data = pd.DataFrame.from_dict(timeseries, orient='index')
        data[symbol] = data['5. adjusted close'].astype(float)
        data = data.drop(columns=['1. open','2. high','3. low','4. close','5. adjusted close','6. volume','7. dividend amount','8. split coefficient'])
        return data
    except:
        return response.json()['Error Message']

In [127]:
print(getPriceData('GOOG'))

                 GOOG
2004-08-19    49.9827
2004-08-20    53.9528
2004-08-23    54.4957
2004-08-24    52.2392
2004-08-25    52.8021
2004-08-26    53.7535
2004-08-27    52.8768
2004-08-30    50.8145
2004-08-31    50.9939
2004-09-01    49.9378
2004-09-02    50.5655
2004-09-03    49.8183
2004-09-07    50.6003
2004-09-08    50.9590
2004-09-09    50.9640
2004-09-10    52.4683
2004-09-13    53.5493
2004-09-14    55.5368
2004-09-15    55.7909
2004-09-16    56.7722
2004-09-17    58.5256
2004-09-20    59.4571
2004-09-21    58.7000
2004-09-22    58.9690
2004-09-23    60.1844
2004-09-24    59.6913
2004-09-27    58.9092
2004-09-28    63.1931
2004-09-29    65.2953
2004-09-30    64.5580
...               ...
2019-09-26  1241.3900
2019-09-27  1225.0900
2019-09-30  1219.0000
2019-10-01  1205.1000
2019-10-02  1176.6300
2019-10-03  1187.8300
2019-10-04  1209.0000
2019-10-07  1207.6801
2019-10-08  1189.1300
2019-10-09  1202.3101
2019-10-10  1208.6700
2019-10-11  1215.4500
2019-10-14  1217.1400
2019-10-15

In [128]:
price2 = pd.read_sql('select * from GOOG', cnx,index_col='index')
price2.head()

,GOOG
index,
2004-08-19,49.9827
2004-08-20,53.9528
2004-08-23,54.4957
2004-08-24,52.2392
2004-08-25,52.8021


In [206]:
def getAllPriceData(symbols,start_date,end_date):
    price = pd.DataFrame()
    for symbol in symbols:
        if(checkSymbol(symbol) == 1):
            price = pd.merge(price,getPriceData(symbol),how='outer',left_index=True,right_index=True)
        else:
            return 'symbol '+symbol+' not found'
    price = price.loc[start_date:end_date]
    return price

In [207]:
start_date = '2014-10-01'
end_date = '2019-11-01'
price = price.loc[start_date:end_date]
price.head()

,AAPL,MSFT,GOOG
index,,,
2014-10-01,91.0073,41.0882,566.7141
2014-10-02,91.6679,40.9629,568.5191
2014-10-03,91.4110,41.2583,573.7049
2014-10-06,91.4110,41.2583,575.7692
2014-10-07,90.6127,40.7570,562.1965


In [208]:
symbols = ['GOOGFG']
start_date = '2014-10-01'
end_date = '2019-11-01'
getAllPriceData(symbols,start_date,end_date)

'symbol GOOGFG not found'